In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os

In [2]:
df = pd.read_csv(Path('/Users/daksh/Desktop/MLOps/emotion_detection/data/interim/train_processed.csv'))

In [3]:
df

,sentiment,content
0,8,thundershower plus baseball equal awwww
1,4,hangover movie gonna hilarious wish could see ...
2,12,playing game leave work work til least pm tonight
3,6,stick work till freakin am madd suck worked day
4,11,sarah x atl u mean jack barakat s wow u ever g...
...,...,...
31995,8,cleaning
31996,6,ooo goddddd vodafone trouble network
31997,4,watching run fat boy run haha soo funny
31998,6,calvinharris make pork product can they ve fuc...


In [4]:
df['content'] = df['content'].fillna('')

In [5]:
df['sentiment'].value_counts()

sentiment
8     6898
12    6793
5     4181
10    4119
7     3080
11    1762
4     1438
9     1174
6     1055
2      665
3      596
1      148
0       91
Name: count, dtype: int64

In [6]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [7]:

vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import dagshub
dagshub.init(repo_owner='dakshvandanarathi', repo_name='Emotion-Detection', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/dakshvandanarathi/Emotion-Detection.mlflow')

mlflow.set_experiment('Logistic Regression Baseline Model')

Accessing as dakshvandanarathi

Initialized MLflow to track repo "dakshvandanarathi/Emotion-Detection"

Repository dakshvandanarathi/Emotion-Detection initialized!

In [12]:
with mlflow.start_run():
    
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")

    # Save and log the notebook
    notebook_path = "exp1_baseline_model.ipynb"
    # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

/Users/daksh/anaconda3/envs/emotion_env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2024/10/01 02:55:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Accuracy: 0.335
Precision: 0.30951815560043433
Recall: 0.335
F1 Score: 0.30611112125543005


2024/10/01 02:55:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run marvelous-smelt-282 at: https://dagshub.com/dakshvandanarathi/Emotion-Detection.mlflow/#/experiments/0/runs/c1bcc08c85a44d358f197fbb532da523.
2024/10/01 02:55:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/dakshvandanarathi/Emotion-Detection.mlflow/#/experiments/0.
